In [20]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import matplotlib.pyplot as plt

In [21]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
print(device)

cpu


In [23]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [24]:
#Dataloader

#Path for training and testing directory
train_path='C:/Users/black/Desktop/FIFTH SEM/mini project/gender_train'
test_path='C:/Users/black/Desktop/FIFTH SEM/mini project/gender_test'
#predict_path='C:/Users/black/Desktop/FIFTH SEM/mini project/gender_predict'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [25]:
#Dataloader

#Path for training and testing directory
train_path_1='C:/Users/black/Desktop/FIFTH SEM/Size/size_train'
test_path_1='C:/Users/black/Desktop/FIFTH SEM/Size/size_test'
#predict_path='C:/Users/black/Desktop/FIFTH SEM/mini project/gender_predict'

train_loader_1=DataLoader(
    torchvision.datasets.ImageFolder(train_path_1,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader_1=DataLoader(
    torchvision.datasets.ImageFolder(test_path_1,transform=transformer),
    batch_size=32, shuffle=True
)

In [26]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [27]:
#categories
root=pathlib.Path(train_path_1)
classes_1=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [28]:
print(classes)

['men', 'women']


In [29]:
print(classes_1)

['size10', 'size11', 'size5', 'size7', 'size8']


In [30]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)

        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)  
        output=self.pool(output)   
        output=self.conv2(output)
        output=self.relu2(output)    
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)    
        output=output.view(-1,32*75*75)   
        output=self.fc(output)   
        return output
            
        


In [31]:
#Optmizer and loss function
model = ConvNet(num_classes=6)
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [32]:
#Optmizer and loss function
model_1= ConvNet(num_classes=6)
optimizer_1=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function_1=nn.CrossEntropyLoss()

In [33]:
num_epochs=30


In [34]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.tiff'))
test_count=len(glob.glob(test_path+'/**/*.tiff'))


In [35]:
#calculating the size of training and testing images
train_count_1=len(glob.glob(train_path_1+'/**/*.tiff'))
test_count_1=len(glob.glob(test_path_1+'/**/*.tiff'))

In [36]:
print(train_count,test_count)

1150 340


In [37]:
print(train_count_1,test_count_1)

699 285


In [ ]:
best_accuracy = 0.0
train_loss_values = []  # Define an empty list to store training loss values

for epoch in range(num_epochs):
    
    # Evaluation and training on the training dataset
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)
        
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    train_loss_values.append(train_loss)  # Append training loss to the list
    
    # Evaluation on the testing dataset
    model.eval()
    
    test_accuracy = 0.0
    for i, (images, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
    
    test_accuracy = test_accuracy / test_count
    
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) + ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint_gender.model')
        best_accuracy = test_accuracy
        
plt.figure(figsize=(10, 5))
plt.plot(range(len(train_loss_values)), train_loss_values, label='Training Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
